<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

This notebook is modified based on https://github.com/microsoft/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [15]:
%%capture --no-stderr
# %pip install pyautogen~=0.1.0

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [20]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
# config_list_gpt35 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": {
#             "gpt-3.5-turbo",
#             "gpt-3.5-turbo-16k",
#             "gpt-3.5-turbo-0301",
#             "chatgpt-35-turbo-0301",
#             "gpt-35-turbo-v0301",
#         },
#     },
# )

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

In [23]:
llm_config = {"config_list": config_list_gpt4, "seed": 42}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="TERMINATE"
)
sim = autogen.AssistantAgent(
    name="Simulated_world",
    system_message="updates the state of the construction site based upon what actions everyone takes, based on what is most likely in the situation on site. is always very brief",
    llm_config=llm_config,
)
constr = autogen.AssistantAgent(
    name="Construction_Manager",
    system_message="responsible for the smooth running of the site ,ultimately accountable for site safety .pays close attention to the constructability of the design, and to the logistics of movements in and out of site.",
    llm_config=llm_config,
)
contractor = autogen.AssistantAgent(
    name="Main_Building_contractor",
    system_message="holds the main contract for constructing the site, including the concrete pouring and the swimming pool",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, sim, constr,contractor], messages=[], max_round=8)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## Start Chat

In [27]:
user_proxy.initiate_chat(
    manager,
    message="""Problem: you are building a new leisure centre. It is a wet and windy day and the concrete wagon has slipped in mud and has fallen on one side . Everyone is unhurt, but you need to reset operations so normal construction schedule can resume tomorrow one day late""",
)

User_proxy (to chat_manager):

Problem: you are building a new leisure centre. It is a wet and windy day and the concrete wagon has slipped in mud and has fallen on one side . Everyone is unhurt, but you need to reset operations so normal construction schedule can resume tomorrow one day late

--------------------------------------------------------------------------------
Construction_Manager (to chat_manager):

Actions:

1. Prioritise Safety: First and foremost, ensure the safety of all workers on site. Make sure no one approaches the fallen wagon and cordon off the area.

2. Report the Incident: Log the incident in the site’s accident report log and notify upper management, as well as relevant safety officers and insurers.

3. Organise Recovery: Contact a recovery service to handle the fallen concrete wagon. Arrange for a crane or specialty vehicle recovery to upright the truck and remove it safely from the site.

4. Assess Damage: Conduct a thorough assessment of the site to ascert